In [1]:
!pip install icecream

In [2]:
from fairtorch import ConstraintLoss, DemographicParityLoss, EqualiedOddsLoss

import random
import numpy as np 
import os
import torch 
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

from icecream import ic
import math

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

### Prepare synthetic dataset 

In [4]:
n_samples = 512
n_feature = 4 

def generate_data(n_samples=n_samples, n_feature=n_feature):
    
    y = np.random.randint(0,2, size=n_samples)
    loc0 = np.random.uniform(-1,1,n_feature)
    loc1 = np.random.uniform(-1,1,n_feature)
    
    X = np.zeros((n_samples, n_feature))
    for i,u in enumerate(y):
        if y[i]==0:
            X[i] = np.random.normal(loc=loc0,scale=1.0, size=n_feature)
        else:
            X[i] = np.random.normal(loc=loc1, scale=1.0, size=n_feature)
    
    sensitive_feature = (X[:,0]>X[:,0].mean()).astype(int)
 
    return X, y, sensitive_feature

In [5]:
dataset = generate_data(1024,4)

#### split dataset

In [6]:
n_train = int(0.7 * len(dataset[0]))
X_train, y_train, sensitive_train = map(lambda x: x[:n_train], dataset)
X_test, y_test, sensitive_test = map(lambda x: x[n_train:], dataset)

In [7]:
print('TRAIN')
display(X_train.shape, y_train.shape, sensitive_train.shape)
print('TEST')
display(X_test.shape, y_test.shape, sensitive_test.shape)

TRAIN


(716, 4)

(716,)

(716,)

TEST


(308, 4)

(308,)

(308,)

In [8]:
class MyDataset:
    def __init__(self,features,targets, sensitive):
        self.features = features
        self.targets = targets
        self.sensitive = sensitive
    def __len__(self):
        return (len(self.features))
    def __getitem__(self,idx):
        dct = {
            'x': torch.tensor(self.features[idx,:],dtype=torch.float),
            'y': torch.tensor(self.targets[idx],dtype=torch.float),
            's': torch.tensor(self.sensitive[idx],dtype=torch.float)
        }
        return dct

In [9]:
class SimpleModel(nn.Module):
    def __init__(self, n_features):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(n_features,1)
    def forward(self,inputs):
        outputs = self.fc(inputs)
        return outputs
    

In [35]:
n_epochs = 50
batch_size = 3
n_feature = 4

model = SimpleModel(n_feature)
criterion = nn.BCEWithLogitsLoss()
dp_criterion = DemographicParityLoss(sensitive_classes=[0,1], alpha=100)
optimizer = optim.SGD(model.parameters(), lr=.1)

for param in model.parameters():
    nn.init.normal_(param, 0, 1e-7)

# train
model.train()

train_dataset = MyDataset(X_train, y_train, sensitive_train)
test_dataset = MyDataset(X_test, y_test, sensitive_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


for data in train_loader:
    inputs, targets, sensitive = data['x'], data['y'], data['s']
    outputs = model(inputs)
    dp_loss = dp_criterion(inputs,outputs,sensitive).item()

    if math.isnan(dp_loss):
        ic(dp_loss,inputs.shape)
        break
    
print('FINISH')




ic| dp_loss: nan, inputs.shape: torch.Size([3, 4])


FINISH


In [11]:
n_epochs = 50
batch_size = 32
n_feature = 4

model = SimpleModel(n_feature)
criterion = nn.BCEWithLogitsLoss()
dp_criterion = DemographicParityLoss(sensitive_classes=[0,1], alpha=100)
optimizer = optim.SGD(model.parameters(), lr=.1)

for param in model.parameters():
    nn.init.normal_(param, 0, 1e-7)

# train
model.train()

train_dataset = MyDataset(X_train, y_train, sensitive_train)
test_dataset = MyDataset(X_test, y_test, sensitive_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


for data in train_loader:
    inputs, targets, sensitive = data['x'], data['y'], data['s']
    outputs = model(inputs)
    dp_loss = dp_criterion(inputs,outputs,sensitive).item()

    if math.isnan(dp_loss):
        ic(dp_loss,inputs.shape)
        break
print("FINISH")

FINISH


In [12]:
!python --version

Python 3.8.6


In [13]:
!pip list -v

Package                       Version             Location                               Installer
----------------------------- ------------------- -------------------------------------- ---------
alembic                       1.4.3               /opt/conda/lib/python3.8/site-packages conda
argon2-cffi                   20.1.0              /opt/conda/lib/python3.8/site-packages conda
asttokens                     2.0.4               /opt/conda/lib/python3.8/site-packages pip
async-generator               1.10                /opt/conda/lib/python3.8/site-packages pip
attrs                         20.3.0              /opt/conda/lib/python3.8/site-packages conda
backcall                      0.2.0               /opt/conda/lib/python3.8/site-packages conda
backports.functools-lru-cache 1.6.1               /opt/conda/lib/python3.8/site-packages conda
bleach                        3.2.1               /opt/conda/lib/python3.8/site-packages conda
blinker                       1.4             